# [Fine-tuning YOLOv8 nano for License Plate Recognition](https://www.youtube.com/watch?v=9KnARRqbkzY)

---

This notebook demonstrates how to use a YOLO model from Roboflow for license plate recognition.


Before running a notebook for the first time, install the required dependencies with:

```bash
	make requirements-nb
```


In [1]:
from civicpulseML.config import BASE_DIR, CUDA_AVAILABLE, MODELS_DIR, RAW_DIR

## Importing dataset from Roboflow

---


In [2]:
from civicpulseML.dataset import download_roboflow_dataset

### RoboFlow API


Add your Roboflow API key to a `.env` file in the project root:

```
ROBOFLOW_API_KEY=your_api_key_here
```


### RoboFlow dataset download


In [3]:
dataset = download_roboflow_dataset(
	workspace="roboflow-universe-projects",
	project_name="license-plate-recognition-rxg4e",
	version=11,
    format_type="yolov8",
    output_dir=RAW_DIR / "roboflow-license-plates"
)

loading Roboflow workspace...
loading Roboflow project...


## Fine-Tuning Pretrained YOLOv8 Model

---

Now, we can fine-tune a pretrained YOLOv8 model on our dataset.


In [4]:
from pathlib import Path
import shutil

from ultralytics import YOLO

### Defining paths


In [5]:
# Creating directory to store final YOLO model weights

yolo_path = MODELS_DIR / "yolov8n-roboflow-license-plates"
yolo_path.mkdir(parents=True, exist_ok=True)

### Load pretrained YOLOv8n


In [6]:
model = YOLO("yolov8n.pt")	# YOLO downloads model to cwd

### Training model & saving weights


In [7]:
# Train the model on the Roboflow dataset

# Check if yolo_path directory contains any items
if yolo_path.exists() and any(yolo_path.iterdir()):
	print(f"yolo_path {yolo_path} contains files; will attempt to use existing weights if applicable.")
else:
	print(f"yolo_path {yolo_path} is empty or missing; training will start from base weights.")

	results = model.train(
		data=dataset.location + "/data.yaml",
		# Use dataset path from Roboflow
		epochs=5,
		imgsz=640,
		batch=16,
		workers=2,
		device='cuda' if CUDA_AVAILABLE else 'cpu'
	)
	
	# Copy weights to a safe folder
	shutil.copy(
		BASE_DIR / "runs/detect/train/weights/best.pt",
		yolo_path / "yolov8n_roboflow_license_plates_best.pt"
	)
	shutil.copy(
		BASE_DIR / "runs/detect/train/weights/last.pt",
		yolo_path / "yolov8n_roboflow_license_plates_last.pt"
	)
	print("Weights saved to ", yolo_path)

	# Delete unnecessary interim run directory
	shutil.rmtree(BASE_DIR / "runs")

yolo_path /home/genesis/CS/civicpulse-ml-dev/models/yolov8n-roboflow-license-plates contains files; will attempt to use existing weights if applicable.


In [8]:
# Delete downloaded pre-trained yolo files
for pt_file in Path(".").glob("*.pt"):
	pt_file.unlink()